# Ollama Webpage summarization
 Before executing the code, ensure that the initial setup outlined in the Readme.md file has been completed.

In [55]:
# Import
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [56]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"   # 11434 port where ollama runs
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"  #Model type

### Test: Verifying Connection with Ollama

In [57]:
# Create a messages list. Has the same format  used on OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }



There is 2 ways of running ollama package:

- Direct HTTP call: you can send HTTP POST requests to the Ollama API
- ollama python package: provides a higher-level interface to interact with the model, abstracting the direct HTTP requests
- Using OpenAI python library to connect to Ollama

This are making web requests locally from my box to my box. And it's connecting to the llama 3.2 model that's being served by llama, and running at localhost:11434


<u> Recommendation:</u>
- If you are primarily using Ollama: Go with ollama Python Package. It’s the cleanest and most idiomatic solution specifically designed for Ollama.
- If you are using both OpenAI and Ollama APIs: Use OpenAI Library (OpenAI) with custom configuration. This provides a unified approach to handling both APIs.
- For advanced or custom needs: Use Direct HTTP Calls (requests.post) for full control and flexibility, especially if you need to customize headers, handle retries, or debug raw API responses.

In [58]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, blog posts, social media posts, and even entire books. This can help businesses save time and resources on content creation.
2. **Marketing Automation**: Generative AI can be used to create personalized marketing campaigns by generating product descriptions, email subject lines, and even entire advertisements.
3. **Product Design**: Generative AI can be used to design new products and product prototypes. For example, it can generate 3D models of products, or even entire product lines.
4. **Customer Service Chatbots**: Generative AI can be used to create chatbots that can respond to customer inquiries in a more personalized and empathetic way.
5. **Financial Analysis**: Generative AI can be used to analyze financial data and generate insights on trends, patterns, and predictions.
6. **Pr

In [59]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: AI-generated content such as articles, social media posts, and product descriptions can help businesses save time and resources while maintaining consistency and quality.
2. **Marketing Automation**: Generative AI can be used to generate personalized marketing messages, personalize customer experiences, and automate repetitive tasks, such as email templates and social media posts.
3. **Product Design**: AI-generated designs for products such as fashion items, electronics, and home decor can help businesses reduce design costs and improve product development speed.
4. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, answer frequently asked questions, and route complex issues to human representatives.
5. **Data Analysis and Visualization**: AI-generated data visualizations can help businesses analyze large datasets, identify patt

In [60]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to create high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and money on content creation while maintaining a consistent tone and style.
2. **Chatbots and Virtual Assistants**: Generative AI-powered chatbots can provide personalized customer support, answer frequently asked questions, and even engage in conversations to build brand loyalty.
3. **Image and Video Generation**: Generative AI can be used to create realistic images and videos for various applications such as advertising, marketing, and entertainment.
4. **Product Design and Development**: Generative AI can help designers generate new product ideas, optimize existing designs, and even prototype products quickly and efficiently.
5. **Recommendation Systems**: Generative AI-powered recommendation syst

### Project

In [61]:
#Prompt Setup:

"""We need to create a structure the APIs understand:
#[ {"role": "system", "content": "system message goes here"},
#  {"role": "user", "content": "user message goes here"} ] """


system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


def user_prompt_for(website): # A function that writes a User Prompt that asks for summaries of websites:
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt


def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]



In [62]:
# Web Scraping 


class Website: # A class to represent a Webpage, analyize the webpage and extract the information

    def __init__(self, url): # Create this Website object from the given url using the BeautifulSoup librarY
        
        self.url = url

        try: #Handling errors
            response = requests.get(url)
            response.raise_for_status()  # Raises HTTPError if the response code is not 200
            soup = BeautifulSoup(response.content, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            for irrelevant in soup.body(["script", "style", "img", "input"]): #Clean some unseful things
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching the URL: {e}")
            return None


In [63]:

#Website Class Testing

try:
    law = Website("https://www.law.com/?slreturn=20250125165349")
    print(law.title)
    print(law.text)
    print(user_prompt_for(law))
    messages_for(law) #Test the summary of the webpage

except Exception as e:
    print(f"Error initializing Website object: {e}")



Law.com | The Premier Source for Global Legal News & Analysis
Newsletters
Events
Sign In
United States
International
MENU
SEARCH
My Law.com
NEW
Sign In
Law.com
Homepage
About Us
Events
Publications
Topics
Insight & Analysis
Surveys & Rankings
Law Firms
Practice Tools & Resources
Communities
Public Notices
All Sections
Membership programs
Law.com Pro
Law.com Pro Mid-Market
Global Leaders in Law
Global Leaders in Law Advisers
Private Client Global Elite
Media brands
Law.com
Law.com Radar
American Lawyer
Corporate Counsel
National Law Journal
Legal Tech News
New York Law Journal
The Legal Intelligencer
The Recorder
Connecticut Law Tribune
Daily Business Review
Daily Report
Delaware Business Court Insider
New Jersey Law Journal
Texas Lawyer
Supreme Court Brief
Litigation Daily
CLOSE
Membership programs
Law.com Pro
Law.com Pro Mid-Market
Global Leaders in Law
Global Leaders in Law Advisers
Private Client Global Elite
Media brands
Law.com
Law.com Radar
American Lawyer
Corporate Counsel
Natio

In [64]:
# Model Execution
import ollama

def summarize(url):  # And now: call the  API, to do the summarization
    website = Website(url) # Web scraping
    try:
        response = ollama.chat(model=MODEL, messages=messages_for(website))  # LLM inference
        return response['message']['content']
    except Exception as e:
        print(f"Error during LLM API call: {e}")
        return None



summarize("https://www.law.com/?slreturn=20250125165349")


"This appears to be a news aggregator website or a platform that collects and summarizes articles from various legal sources. Here's a brief overview:\n\n**Content:**\n\nThe platform aggregates news articles, summaries, and analysis on various topics related to the law, including:\n\n1. Law firm news and management\n2. Legal education and training\n3. Diversity and inclusion\n4. Technology and AI in law\n5. Litigation and arbitration\n6. In-house counsel and corporate legal issues\n\n**Features:**\n\nThe platform offers various features, such as:\n\n1. **Radar**: a system that tracks key developments and trends in the legal industry.\n2. **Legal Compass**: a tool for finding local networking opportunities and law firms.\n3. **Data-driven reporting**: the platform uses ALM's proprietary resources to provide data-driven insights.\n4. **Events and webcasts**: the platform hosts various events, webcasts, and conferences related to the law.\n\n**Target audience:**\n\nThe target audience app

In [65]:

def display_summary(url):# A function to display this nicely in the Jupyter output, using markdown
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://www.law.com/?slreturn=20250125165349")

This text appears to be a sample issue of the ALM (American Law Media) publication "Law.com", which is a media platform offering competitive intelligence and insights on the legal industry. The content includes:

1. News articles:
	* A Skadden team joins White & Case in Mexico City for a Citigroup demerger.
	* Lawyers take different approaches to handling wildfires in California.
	* A court rules that a poop-themed dog toy is parody, but still damages Jack Daniel's brand.
2. In-house insights:
	* The L Suite acquires Luminate+, a legal e-learning platform.
	* Gen AI and Davis Wright Tremaine develop a training model for associate legal writing.
3. Events and webcasts:
	* Practical in-house insights on activating AI in law firms.
	* A survey of firm leaders setting strategy for 2025.
4. Litigation news:
	* A possible owner of the historic Miami courthouse is identified amid delays on a new building.
5. Popular topics:
	* DEI rollbacks and their impact on employment attorneys.
	* Diversity in the legal industry, including a discussion on corporate commitment to equality.
6. Law firm management:
	* How Morris Nichols became a major player in China.
7. Young professionals:
	* A Paul Weiss associate's career took off with help from a social mobility alliance.
8. Resources and articles:
	* AI-powered deposition and medical record summaries.
	* Aligning client needs with lawyer growth and profitability.
9. Promotions and advertising.

The text also includes various calls-to-action, such as signing up for Law.com or registering for events and webcasts.